In [6]:
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, DallETool


# ────────────────────────────────────────────────────────────────────────────────
# Variables - these can be changed:
# ────────────────────────────────────────────────────────────────────────────────

verbose_agents = True # A value of `True` (recommended) will instruct the Agent to output information while it is in progress; or `False` to get no information and just the result.


# ────────────────────────────────────────────────────────────────────────────────
# Initialize constants - please don't change these values:
# ────────────────────────────────────────────────────────────────────────────────

load_dotenv()

scrape_website_tool = ScrapeWebsiteTool()

geocoding_url = "https://geocoding-api.open-meteo.com/v1/search?name={{location}}"


# ────────────────────────────────────────────────────────────────────────────────
# Build the agents
# ────────────────────────────────────────────────────────────────────────────────
producer = Agent(
    role="Travel Agent Producer",
    goal="Produce, using your Crew, a written article about a location, with accompanying images.",
    backstory="""You are an expert producer and servant leader, skilled in gathering input from a human user and overseeing complex projects and guiding teams to success.
                 You can only ask for input from the human user. You cannot do any other task. You always must delegate other tasks to your crew.""",
    allow_delegation=True,
    verbose=verbose_agents,
    llm="openai/gpt-4o",
)

user_surveyor = Agent(
    role="User Surveyor",
    goal="Ask questions to the human user, and return the response.",
    backstory="You are an expert communicator. You ask for input from the human user and return their response as plain text.",
    verbose=verbose_agents,
)

geocoder = Agent(
    role="Geocoder",
    goal="Geocode a location in to a latitude and longitude, using a web API.",
    backstory=f"""You have perfected the ability to geocode a location in to a latitude and longitude, from the specified Geocoding URL.
                  You will need to pass the location in-place of the {{location}} placeholder in the specified Geocoding URL, before you call the URL.
    
                  Geocoding URL: {geocoding_url}

                  From the results of calling the Geocoding URL, get the latitude and longitude from the first item in the United Kingdom.
                  You are expected to return the result in JSON format.
                  """,
    tools=[scrape_website_tool],
    verbose=verbose_agents,
    llm="openai/o4-mini",
)


# ────────────────────────────────────────────────────────────────────────────────
# Define the tasks
# ────────────────────────────────────────────────────────────────────────────────
test = Task(
    description="""Ask the user to input a location in the United Kingdom and then geocode the location.""",
    expected_output="The geocoded location that the user inputted.",
    agent=producer,
    human_input=True,
)


# ────────────────────────────────────────────────────────────────────────────────
# Demo run
# ────────────────────────────────────────────────────────────────────────────────
result = Crew(
    agents=[
        producer,
        user_surveyor,
        geocoder,
    ],
    tasks=[
        test,
    ],
).kickoff()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Agent Producer                                                                                   │
│                                                                                                                 │
│  Task: Ask the user to input a location in the United Kingdom and then geocode the location.                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: User Surveyor                                                                                           │
│                                                                                                                 │
│  Task: Can you please provide a location in the United Kingdom for us to work on?                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: User Surveyor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Could you please provide me with a specific location in the United Kingdom that you would like us to work on   │
│  for geocoding? This will help us move forward effectively with our task.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Agent Producer                                                                                   │
│                                                                                                                 │
│  Thought: Thought: I need to ask the user to input a location in the United Kingdom.                            │
│                                                                                                                 │
│  Using Tool: Ask question to coworker                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"question\": \"Can you please provide a location in the United Kingdom for us to work on?\", \"context\":   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Agent Producer                                                                                   │
│                                                                                                                 │
│  Thought: Thought: The user has provided the location "Nottingham" in the United Kingdom. I need to delegate    │
│  the task of geocoding this location.                                                                           │
│                                                                                                                 │
│  Using Tool: Delegate work to coworker                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"task\": \"Geocode the location provided by the user.\", \"context\": \"The user has provided the location  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {"latitude":52.954783,"longitude":-1.158109}                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Agent Producer                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The geocoded location for Nottingham, United Kingdom is latitude 52.954783 and longitude -1.158109.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 ## Final Result: The geocoded location for Nottingham, United Kingdom is latitude 52.954783 and longitude -1.158109.
 

=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement requests.
 - You can provide multiple rounds of feedback until satisfied.
=====

